In [1]:
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import json

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

## Instantiate Spotify Session

Creating connection so spotifies database to extract song and playlist data

In [2]:
#extract credentials from environment variabes and instantiate session
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [13]:
#grab playlists for user
myPlaylists = sp.user_playlists("ben.doan4366")

#extract tracks for gym and cooking playlists
gymPlaylist = sp.playlist_tracks("1Vu8EHOlNPBFVBxmKtGHlr")
cookingPlaylist = sp.playlist_tracks("5myx61jwlbF2iMDVWNK0Ps")
codingPlaylist = sp.playlist_tracks("1V9wButKEb5GJDshdKUGeN")

## Design Helper Functions to Transform Raw Data to Staging Table Format

In [4]:
#function to transition raw json response from Spotify to dataframe of artist and so

def prepArtistStagingTables(raw_playlist_dictionary):
    
    #trim meta data and extract track field from raw dictionary
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    
    #generate raw dataframe
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    #extract artist column from pd
    artist_df = playlist_df_raw.loc[:,["name","artists"]].explode("artists")
    
    #extract artist dictionary values into series, and recast to dataframe 
    artist_fields_expanded = artist_df["artists"].apply(pd.Series)
    artist_fields_expanded.rename(columns={"name":"artist_name"}, inplace=True)
    
    #concat recasted dataframe to original dataframe with song name
    #artist_df_final = pd.concat([artist_df, artist_fields_expanded], axis=1)
    artist_df_final = pd.concat([artist_df, artist_fields_expanded], axis=1).loc[:,["name", "id", "artist_name", "type", "uri"]]
    artist_df_final.rename(columns={"name":"song_name"}, inplace=True)
    artist_df_final.rename(columns={"id":"artist_id"}, inplace=True)
    
    return artist_df_final

In [14]:
def prepAlbumStagingTables(raw_playlist_dictionary):
    #trim meta data and extract item field from raw dictionary
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    
    #generate raw dataframe
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    
    #extract artist column from pd
    album_df = playlist_df_raw.loc[:,["name", "album"]]
    album_df_serialized = album_df["album"].apply(pd.Series)
    album_df_serialized.rename(columns={"name":"album_name"}, inplace=True)
    album_df_serialized.rename(columns={"id":"album_id"}, inplace=True)
    album_df_serialized.rename(columns={"uri":"album_uri"}, inplace=True)

    #extract artist dictionary values into series, and recast to dataframe 
    album_with_artists_df = album_df_serialized.explode("artists")
    album_with_artists_df_serialized = album_with_artists_df["artists"].apply(pd.Series)
    album_with_artists_df_serialized.rename(columns={"name":"artist_name"}, inplace=True)
    album_with_artists_df_serialized.rename(columns={"id":"artist_id"}, inplace=True)

    
    #concat recasted dataframe to original dataframe with song name
    album_df_final = pd.concat([album_with_artists_df_serialized, album_df_serialized, album_df], axis=1)
    album_df_final.rename(columns={"name":"song_name"}, inplace=True)
    album_df_final.rename(columns={"id":"album_id"}, inplace=True)
    
    return album_df_final.loc[:,["album_name", "album_id", "artist_name", "artist_id","song_name","release_date", "album_uri"]]

In [16]:
def generate_song_nodes(raw_playlist_dictionary):
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    song_nodes = playlist_df_raw.loc[:,["name", "id", "uri","duration_ms","explicit","popularity"]]
    song_nodes.rename(columns={"name":"song_name"}, inplace=True)
    song_nodes.rename(columns={"id":"song_id"}, inplace=True)
    
    return song_nodes

In [17]:
def generate_artist_nodes(artist_df):
    artist_nodes = artist_df.loc[:,["artist_name", "artist_id"]]
    #artist_nodes.drop_duplicates(["artist_id"], inplace=True)
    
    return artist_nodes

In [18]:
def generate_album_nodes(album_df):
    album_nodes = album_df.loc[:,["album_name", "album_id"]]
    #album_nodes.drop_duplicates(["album_id"], inplace=True)
    
    return album_nodes

## Create Staging Tables

##### Extract data from cooking and gym playlists into staging table for artist-to-song edges

In [19]:
cooking_playlist_artists_df = prepArtistStagingTables(cookingPlaylist)
gym_playlist_artists_df = prepArtistStagingTables(gymPlaylist)
coding_playlist_artists_df = prepArtistStagingTables(codingPlaylist)

full_playlist_artists_df = pd.concat([cooking_playlist_artists_df, gym_playlist_artists_df, coding_playlist_artists_df])

##### Extract data from cooking and gym playlists into staging Table for Album to song edges

In [20]:
cooking_playlist_albums_df = prepAlbumStagingTables(cookingPlaylist)
gym_playlist_albums_df = prepAlbumStagingTables(gymPlaylist)
coding_playlist_albums_df = prepAlbumStagingTables(codingPlaylist)

full_album_playlist_df = pd.concat([cooking_playlist_albums_df, gym_playlist_albums_df, coding_playlist_albums_df])

##### Create Staging Tables for Song nodes

In [22]:
gym_song_nodes = generate_song_nodes(gymPlaylist)
gym_song_nodes["playlist_id"] = "1Vu8EHOlNPBFVBxmKtGHlr"
gym_song_nodes["playlist_name"] = "brrrat brrrat"

cooking_playlist_nodes = generate_song_nodes(cookingPlaylist)
cooking_playlist_nodes["playlist_id"] = "5myx61jwlbF2iMDVWNK0Ps"
cooking_playlist_nodes["playlist_name"] = "wrist whippin"

coding_playlist_nodes = generate_song_nodes(codingPlaylist)
coding_playlist_nodes["playlist_id"] = "1V9wButKEb5GJDshdKUGeN"
coding_playlist_nodes["playlist_name"] = "werk"

all_song_nodes = pd.concat([gym_song_nodes, cooking_playlist_nodes, coding_playlist_nodes])
all_song_nodes

,song_name,song_id,uri,duration_ms,explicit,popularity,playlist_id,playlist_name
0,Fight Night,2n5gVJ9fzeX2SSWlLQuyS9,spotify:track:2n5gVJ9fzeX2SSWlLQuyS9,216247,True,68,1Vu8EHOlNPBFVBxmKtGHlr,brrrat brrrat
1,Pipe It Up,6eBrlbv2HMYcldwjoMWIrC,spotify:track:6eBrlbv2HMYcldwjoMWIrC,206266,True,53,1Vu8EHOlNPBFVBxmKtGHlr,brrrat brrrat
2,"Floyd Mayweather (feat. Travis Scott, Gucci Ma...",5ALc7rbru6QOLGodVSDocc,spotify:track:5ALc7rbru6QOLGodVSDocc,358506,True,57,1Vu8EHOlNPBFVBxmKtGHlr,brrrat brrrat
3,With Them,0tISnxqgVmxqhVghsTi2Rr,spotify:track:0tISnxqgVmxqhVghsTi2Rr,197746,True,58,1Vu8EHOlNPBFVBxmKtGHlr,brrrat brrrat
4,F Cancer (Boosie) [feat. Quavo],1qPK58q90l8KNoWyXsLI2Y,spotify:track:1qPK58q90l8KNoWyXsLI2Y,249639,True,49,1Vu8EHOlNPBFVBxmKtGHlr,brrrat brrrat
...,...,...,...,...,...,...,...,...
36,Greaze Mode,3nG784YlxS4VQOF0qiHKVP,spotify:track:3nG784YlxS4VQOF0qiHKVP,168400,True,64,1V9wButKEb5GJDshdKUGeN,werk
37,Uber Everywhere (feat. Travis Scott),5YCs9rP6ZcMJW6me3QhtWr,spotify:track:5YCs9rP6ZcMJW6me3QhtWr,181107,True,59,1V9wButKEb5GJDshdKUGeN,werk
38,Throw A Fit,7D8aQaRzoi9Qzz5yerVK5b,spotify:track:7D8aQaRzoi9Qzz5yerVK5b,158757,True,65,1V9wButKEb5GJDshdKUGeN,werk
39,That Go! (feat. T-Shyne),4X4cPQicoAfannkUFUXmi9,spotify:track:4X4cPQicoAfannkUFUXmi9,225866,True,71,1V9wButKEb5GJDshdKUGeN,werk


##### Create staging tables for album and artist nodes

In [23]:
all_artist_nodes = generate_artist_nodes(full_playlist_artists_df)
all_album_nodes = generate_album_nodes(full_album_playlist_df)

In [24]:
all_artist_nodes

,artist_name,artist_id
0,The Internet,7GN9PivdemQRKjDt4z5Zv8
1,Feng Suave,73dudJ9j0HStIhJDU8MjMI
2,Jarreau Vandal,6f96znq79wvlknKHHHhtTW
3,Toro y Moi,6O4EGCCb6DoIiR6B1QCQgp
4,Litany,2z6JjrrJKNLilqlx8mlxcc
...,...,...
39,Meek Mill,20sxb77xiYeusSH8cVdatc
39,T-Shyne,14qypx78D3Jd1mm7c2BqQb
40,Future,1RyvyyTE3xzB2ZywiAwp0i
40,Drake,3TVXtAsR1Inumwj472S9r4


In [35]:
album_edges_staging = full_album_playlist_df.merge(all_song_nodes, on=["song_name"])
artist_edges_staging = full_playlist_artists_df.merge(all_song_nodes, on=["song_name"])

album_edges_staging

,album_name,album_id,artist_name,artist_id,song_name,release_date,album_uri,song_id,uri,duration_ms,explicit,popularity,playlist_id,playlist_name
0,Hive Mind,27ThgFMUAx3MXLQ297DzWF,The Internet,7GN9PivdemQRKjDt4z5Zv8,Come Over,2018-07-20,spotify:album:27ThgFMUAx3MXLQ297DzWF,2hPNuVVSV1tqiD2uPlfehz,spotify:track:2hPNuVVSV1tqiD2uPlfehz,321853,True,57,5myx61jwlbF2iMDVWNK0Ps,wrist whippin
1,Feng Suave,4TXLnG7URez82o8xfZif2h,Feng Suave,73dudJ9j0HStIhJDU8MjMI,By the Poolside,2017-10-04,spotify:album:4TXLnG7URez82o8xfZif2h,4XTWiowLNAWfdc3YNq59VU,spotify:track:4XTWiowLNAWfdc3YNq59VU,217116,False,0,5myx61jwlbF2iMDVWNK0Ps,wrist whippin
2,Westside,1ZyxE4rXSfAZwqd6PO7l1N,Jarreau Vandal,6f96znq79wvlknKHHHhtTW,Westside,2018-07-31,spotify:album:1ZyxE4rXSfAZwqd6PO7l1N,4rU2eED2YPnmmrVmLlaYVI,spotify:track:4rU2eED2YPnmmrVmLlaYVI,179629,False,27,5myx61jwlbF2iMDVWNK0Ps,wrist whippin
3,Anything In Return,3xDRuOqakukb1SjHQG4WWc,Toro y Moi,6O4EGCCb6DoIiR6B1QCQgp,How's It Wrong,2013-01-22,spotify:album:3xDRuOqakukb1SjHQG4WWc,3P5iJl5H2nOmWNMsCFuqFs,spotify:track:3P5iJl5H2nOmWNMsCFuqFs,234079,False,31,5myx61jwlbF2iMDVWNK0Ps,wrist whippin
4,4 Track EP,64NBfuKfE49atmxBp4H3ZZ,Litany,2z6JjrrJKNLilqlx8mlxcc,Bedroom,2017-06-07,spotify:album:64NBfuKfE49atmxBp4H3ZZ,3LH6EfPCeySeuymuK2gWAO,spotify:track:3LH6EfPCeySeuymuK2gWAO,186781,False,63,5myx61jwlbF2iMDVWNK0Ps,wrist whippin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Throw A Fit,2kGxRa3ut2AMvo1fdoUeuK,Tinashe,0NIIxcxNHmOoyBx03SfTCD,Throw A Fit,2018-07-27,spotify:album:2kGxRa3ut2AMvo1fdoUeuK,7D8aQaRzoi9Qzz5yerVK5b,spotify:track:7D8aQaRzoi9Qzz5yerVK5b,158757,True,65,1V9wButKEb5GJDshdKUGeN,werk
274,That Go! (feat. T-Shyne),7z2Kbn2GQ9J1EzoT5MULqq,Young Stoner Life,1xr2G8Hlx4QWmT9HaUbmoO,That Go! (feat. T-Shyne),2021-02-12,spotify:album:7z2Kbn2GQ9J1EzoT5MULqq,4X4cPQicoAfannkUFUXmi9,spotify:track:4X4cPQicoAfannkUFUXmi9,225866,True,71,1V9wButKEb5GJDshdKUGeN,werk
275,That Go! (feat. T-Shyne),7z2Kbn2GQ9J1EzoT5MULqq,Young Thug,50co4Is1HCEo8bhOyUWKpn,That Go! (feat. T-Shyne),2021-02-12,spotify:album:7z2Kbn2GQ9J1EzoT5MULqq,4X4cPQicoAfannkUFUXmi9,spotify:track:4X4cPQicoAfannkUFUXmi9,225866,True,71,1V9wButKEb5GJDshdKUGeN,werk
276,That Go! (feat. T-Shyne),7z2Kbn2GQ9J1EzoT5MULqq,Meek Mill,20sxb77xiYeusSH8cVdatc,That Go! (feat. T-Shyne),2021-02-12,spotify:album:7z2Kbn2GQ9J1EzoT5MULqq,4X4cPQicoAfannkUFUXmi9,spotify:track:4X4cPQicoAfannkUFUXmi9,225866,True,71,1V9wButKEb5GJDshdKUGeN,werk


##### Export to CSV for loading into Neo4j

In [26]:
all_album_nodes.to_csv("../data/album_nodes.csv")
all_artist_nodes.to_csv("../data/artist_nodes.csv")
all_song_nodes.to_csv("../data/song_nodes.csv")

album_edges_staging.to_csv("../data/album_edges.csv")
artist_edges_staging.to_csv("../data/artist_edges.csv")

## Professor Nelson's Data

Repeat all the steps above for Professor Nelson's Spotify Playlists

In [27]:
profNelsonPlaylists = sp.user_playlists("vyrvddd99x5ejdq4b7ikuzgst")
profNelsonPlaylists

{'href': 'https://api.spotify.com/v1/users/vyrvddd99x5ejdq4b7ikuzgst/playlists?offset=0&limit=50',
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/1uY4UVKMPbOFj9cdtNc6a6'},
   'href': 'https://api.spotify.com/v1/playlists/1uY4UVKMPbOFj9cdtNc6a6',
   'id': '1uY4UVKMPbOFj9cdtNc6a6',
   'images': [{'height': 640,
     'url': 'https://mosaic.scdn.co/640/ab67616d0000b2730bf8ea3f7755f12f7a45d90eab67616d0000b2734fdb0337978c7464cdfb1783ab67616d0000b2738a29e873f75006aa646db111ab67616d0000b273b065002ec760593ec9dbe305',
     'width': 640},
    {'height': 300,
     'url': 'https://mosaic.scdn.co/300/ab67616d0000b2730bf8ea3f7755f12f7a45d90eab67616d0000b2734fdb0337978c7464cdfb1783ab67616d0000b2738a29e873f75006aa646db111ab67616d0000b273b065002ec760593ec9dbe305',
     'width': 300},
    {'height': 60,
     'url': 'https://mosaic.scdn.co/60/ab67616d0000b2730bf8ea3f7755f12f7a45d90eab67616d0000b2734fdb0337978c7464cdfb1783ab67616

In [28]:
sailing_raw_json = sp.playlist_tracks("1uY4UVKMPbOFj9cdtNc6a6")
bestOfJB_raw_json = sp.playlist_tracks("6pBkmo0Pn8HSjtcudqqGDE")
jbLive_raw_json = sp.playlist_tracks("5QRStOSftF9l9A9Jgi4HlC")

In [29]:
sailing_song_nodes = generate_song_nodes(sailing_raw_json)
bestOfJB_song_nodes = generate_song_nodes(bestOfJB_raw_json)
jbLive_song_nodes = generate_song_nodes(jbLive_raw_json)

sailing_song_nodes["playlist_id"] = "1uY4UVKMPbOFj9cdtNc6a6"
sailing_song_nodes["playlist_name"] = "sailing"

bestOfJB_song_nodes["playlist_id"] = "6pBkmo0Pn8HSjtcudqqGDE"
bestOfJB_song_nodes["playlist_name"] = "Jimmy Buffett Best Of"

jbLive_song_nodes["playlist_id"] = "5QRStOSftF9l9A9Jgi4HlC"
jbLive_song_nodes["playlist_name"] = "Jimmy Buffet: Live"

nelson_all_song_nodes = pd.concat([sailing_song_nodes, bestOfJB_song_nodes, jbLive_song_nodes])

In [30]:
sailing_artists_df = prepArtistStagingTables(sailing_raw_json)
bestOfJB_artists_df = prepArtistStagingTables(bestOfJB_raw_json)
jbLive_artists_df = prepArtistStagingTables(jbLive_raw_json)


nelson_full_playlist_artists_df = pd.concat([sailing_artists_df, bestOfJB_artists_df,jbLive_artists_df])

In [31]:
sailing_album_df = prepAlbumStagingTables(sailing_raw_json)
bestOfJB_album_df = prepAlbumStagingTables(bestOfJB_raw_json)
jbLive_albums_df = prepAlbumStagingTables(jbLive_raw_json)

nelson_full_playlist_albums_df = pd.concat([sailing_album_df, bestOfJB_album_df,jbLive_albums_df])

In [32]:
nelson_all_artist_nodes = generate_artist_nodes(nelson_full_playlist_artists_df)
nelson_all_album_nodes = generate_album_nodes(nelson_full_playlist_albums_df)

nelson_artist_edges_staging = nelson_full_playlist_artists_df.merge(nelson_all_song_nodes, on=["song_name"])
nelson_album_edges_staging = nelson_full_playlist_albums_df.merge(nelson_all_song_nodes, on=["song_name"])

In [33]:
nelson_all_artist_nodes.to_csv("../data/nelson_artist_nodes.csv")
nelson_all_album_nodes.to_csv("../data/nelson_album_nodes.csv")
nelson_all_song_nodes.to_csv("../data/nelson_song_nodes.csv")

nelson_artist_edges_staging.to_csv("../data/nelson_artist_edges.csv")
nelson_album_edges_staging.to_csv("../data/nelson_album_edges.csv")

In [34]:
master_import_file = pd.concat([album_edges_staging,nelson_album_edges_staging])
master_import_file.to_csv("../data/master_import.csv")